# Introduction
In this script, we will generate the model files for the multiple scenarios we would like to simulate, which in this case are:

- Fluvial flooding (Q100) – Baseline
- Fluvial flooding (Q1000) – Baseline
- Fluvial flooding (Q100) – SSP5 2030
- Fluvial flooding (Q1000) – SSP5 2030
- Pluvial flooding (Q100) – Baseline
- Pluvial flooding (Q1000) – Baseline
- Pluvial flooding (Q100) – SSP5 2030
- Pluvial flooding (Q1000) – SSP5 2030
- Pluvial flooding + Fluvial flooding (Q100) – Baseline
- Pluvial flooding + Fluvial flooding (Q1000) – Baseline
- Pluvial flooding + Fluvial flooding (Q100) – SSP5 2030
- Pluvial flooding + Fluvial flooding (Q1000) – SSP5 2030
- Pluvial flooding + Fluvial flooding (Q1000) – SSP5 2050


# Import packages

In [94]:
from hydromt.config import configread
import glob
import os
import yaml

In [95]:
root = '/Users/aprida/Documents/Consulting/Private_sector/Keolis/Model_Alvaro'

# Functions

In [96]:
from pathlib import Path

def convert_paths_to_strings(data):
    if isinstance(data, dict):
        return {key: convert_paths_to_strings(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_paths_to_strings(item) for item in data]
    elif isinstance(data, Path):
        return str(data)
    else:
        return data

# Generate scenario files

In [97]:
# Define scenarios to model

scenarios = [
    'PL_rp100_24H_baseline',
    'PL_rp1000_24H_baseline',
    'PL_rp100_24H_2030_ssp585',
    'PL_rp1000_24H_2030_ssp585',
    'FL_rp100_baseline',
    'FL_rp1000_baseline',
    'FL_rp100_2030_ssp585',
    'FL_rp1000_2030_ssp585',
    'PLFL_rp100_baseline',
    'PLFL_rp1000_baseline',
    'PLFL_rp100_2030_ssp585',
    'PLFL_rp1000_2030_ssp585',
    'PLFL_rp1000_2050_ssp585'
]

# Generate scenario file names
scenario_files = []

for scenario in scenarios:

    flood_type = scenario.split('_')[0]
    rp = scenario.split('_')[1].split('rp')[1]
    ssp = scenario.split('_')[-1]

    if flood_type == 'PL':
        time_horizon = scenario.split('_')[3]
        tmp = f'{flood_type}_rp{rp}_24H_{time_horizon}.csv' if time_horizon == 'baseline' else f'{flood_type}_rp{rp}_24H_{time_horizon}_{ssp}.csv'
    
    elif flood_type == 'FL':
        time_horizon = scenario.split('_')[2]
        tmp = f'{flood_type}_rp{rp}_{time_horizon}.csv' if time_horizon == 'baseline' else f'{type}_rp{rp}_{time_horizon}_{ssp}.csv'
    
    else:
        tmp = []
        flood_type = ['PL', 'FL']
        time_horizon = scenario.split('_')[2]
        for type in flood_type:
            
            if type == 'PL':
                file = f'{type}_rp{rp}_24H_{time_horizon}.csv' if time_horizon == 'baseline' else f'{type}_rp{rp}_24H_{time_horizon}_{ssp}.csv'
                
            else:
                file = f'{type}_rp{rp}_{time_horizon}.csv' if time_horizon == 'baseline' else f'{type}_rp{rp}_{time_horizon}_{ssp}.csv'
            
            tmp.append(file)
            

    scenario_files.append(tmp)

scenario_files


['PL_rp100_24H_baseline.csv',
 'PL_rp1000_24H_baseline.csv',
 'PL_rp100_24H_2030_ssp585.csv',
 'PL_rp1000_24H_2030_ssp585.csv',
 'FL_rp100_baseline.csv',
 'FL_rp1000_baseline.csv',
 'FL_rp100_2030_ssp585.csv',
 'FL_rp1000_2030_ssp585.csv',
 ['PL_rp100_24H_baseline.csv', 'FL_rp100_baseline.csv'],
 ['PL_rp1000_24H_baseline.csv', 'FL_rp1000_baseline.csv'],
 ['PL_rp100_24H_2030_ssp585.csv', 'FL_rp100_2030_ssp585.csv'],
 ['PL_rp1000_24H_2030_ssp585.csv', 'FL_rp1000_2030_ssp585.csv'],
 ['PL_rp1000_24H_2050_ssp585.csv', 'FL_rp1000_2050_ssp585.csv']]

In [98]:
# Create folder to store model scenario files

model_path = os.path.join(root,'model_scenario_files')
os.makedirs(model_path, exist_ok=True)

model_file_path = []
os.makedirs(model_path, exist_ok=True)

# Edit config model files to change scenario files and save files

res_fl = 300 # Resolution of the fluvial flood model (meters)
res_pl = 100 # Resolution of the pluvial flood model (meters)

dtmax_fl = 60 # Maximum time-step of the fluvial flood model (seconds)
dtmax_pl = 30 # Maximum time-step of the pluvial flood model (seconds)

alpha_fl = 0.5 # Limitation of CFL condition (stability)
alpha_pl = 0.2 # Limitation of CFL condition (stability)

for files, scenario in zip(scenario_files, scenarios):

    model_file = configread('model.yml', abs_path=True)
    model_file['root'] = os.path.join(root, scenario)
    
    model_file['region'] = (
        os.path.join(root, 'Inputs', 'active_mask_fluvialpluvial.geojson')
        if 'PLFL' in scenario
        else os.path.join(root, 'Inputs', 'active_mask_fluvial.geojson')
        if 'FL' in scenario
        else os.path.join(root, 'Inputs', 'active_mask_pluvial.geojson')
    )
    
    model_file['mask_active'] = (
        os.path.join(root, 'Inputs', 'active_mask_fluvialpluvial.geojson')
        if 'PLFL' in scenario
        else (
            os.path.join(root, 'Inputs', 'active_mask_fluvial.geojson')
            if 'FL' in scenario
            else os.path.join(root, 'Inputs', 'active_mask_pluvial.geojson')
        )
    )
    model_file['mask_bounds'] = os.path.join(root, 'Inputs', 'outflow_fluvial.geojson') if 'FL' in scenario else os.path.join(root, 'Inputs', 'outflow_pluvial.geojson')
    model_file['res'] = res_fl if 'fluvial' in model_file['mask_active'] else res_pl
    model_file['dtmax'] = dtmax_fl if 'fluvial' in model_file['mask_active'] else dtmax_pl
    model_file['ant_moist'] = 'wet'
    model_file['alpha'] = alpha_pl if 'PL' in scenario else alpha_fl

    
    if 'PLFL' in scenario:
        model_file['bnd'] = os.path.join(root, 'scenarios', files[1])
        model_file['src'] = '/Users/aprida/Documents/Consulting/Private_sector/Keolis/Hydrography/ts_stations/Q_coords_allstations.geojson'
        model_file['precip'] = os.path.join(root, 'scenarios', files[0])

    elif 'FL_' in scenario:
        model_file['bnd'] = os.path.join(root, 'scenarios', files)
        model_file['src'] = '/Users/aprida/Documents/Consulting/Private_sector/Keolis/Hydrography/ts_stations/Q_coords_allstations.geojson'
        model_file.pop('precip')
        
    elif 'PL_' in scenario:
        model_file['precip'] = os.path.join(root, 'scenarios', files)
        model_file.pop('bnd')
        model_file.pop('src')
        
    else:
        print('No pluvial or fluvial flood scenarios were identified.')

    model_file = convert_paths_to_strings(model_file)
    tmp_model_file_path = os.path.join(model_path, scenario + '.yml')
    model_file_path.append(tmp_model_file_path)
    
    with open(tmp_model_file_path, 'w') as yaml_file:
        yaml.dump(model_file, yaml_file)


In [99]:
# Save model file paths to .txt file

text_file_path = os.path.join(model_path, 'model_file_paths.txt')

with open(text_file_path, 'w') as text_file:
    for element in model_file_path:
        text_file.write(f"{element}\n")

models_run_paths = os.path.join(model_path, 'models_run_paths.txt')

with open(models_run_paths, 'w') as text_file:
    for element in scenarios:
        text_file.write(f"{os.path.join(root, element)}\n")